In [75]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pickle

matplotlib.use("pgf")
matplotlib.rcParams.update({
    # "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [120]:
% install python3-tk

UsageError: Line magic function `%pip3` not found.


In [39]:
PATH = '/Users/antoniaboca/vs_code/l46_project/kd-models/citeseer/final_results.pickle'

datasets = ['citeseer', 'cora', 'pubmed']

stats_arr = {}

for dataset in datasets:
    kd_teacher = np.zeros((21, 3), dtype=np.float32)
    kd_f1 = np.zeros((21,3), dtype=np.float32)
    kd_loss = np.zeros((21, 3), dtype=np.float32)

    mi_teacher = np.zeros((21, 3), dtype=np.float32)
    mi_f1 = np.zeros((21, 3), dtype=np.float32)
    mi_loss = np.zeros((21, 3), dtype=np.float32)

    full_f1 = np.zeros(3, dtype=np.float32)
    full_loss = np.zeros(3, dtype=np.float32)

    for i in range(1, 4):
        if i == 1:
            file_path = f'/Users/antoniaboca/vs_code/l46_project/kd-models/{dataset}/final_results.pickle'
        else:
            file_path = f'/Users/antoniaboca/vs_code/l46_project/kd-models/{dataset}-{i}/final_results.pickle'
        with open(file_path, 'rb') as handle:
            stats = pickle.load(handle)

        if i != 1:
            full_f1[i - 1] = stats['full'][0]['f1 score on test set']
            full_loss[i-1] = stats['full'][0]['loss on training set']

        for j, stat in enumerate(stats['kd']):
            kd_teacher[j][i-1] = stat['teacher accuracy']
            kd_f1[j][i-1] = stat['f1 score on test set']
            kd_loss[j][i-1] = stat['loss on training set']
        
        for j, stat in enumerate(stats['mi']):
            mi_teacher[j][i-1] = stat['teacher accuracy']
            mi_f1[j][i-1] = stat['f1 score on test set']
            mi_loss[j][i-1] = stat['loss on training set']
    
    stats_arr[dataset] = {
        'full': (full_f1, full_loss),
        'kd': (kd_teacher, kd_f1, kd_loss),
        'mi': (mi_teacher, mi_f1, mi_loss)
    }


In [40]:
stats_arr['pubmed']['full'][0][0] = 0.752
stats_arr['pubmed']['full'][1][0] =  0.0003575420705601573

stats_arr['citeseer']['full'][0][0] = 0.671
stats_arr['citeseer']['full'][1][0] =  1.791745662689209

stats_arr['cora']['full'][0][0] = 0.7579999999999999
stats_arr['cora']['full'][1][0] =  0.00010437193850520998


In [41]:
stats_arr['pubmed']['full']

(array([0.752, 0.742, 0.758], dtype=float32),
 array([3.5754207e-04, 1.9446012e-05, 8.4952562e-06], dtype=float32))

In [53]:
# Iteration 0  Cora | Graph sparsity 100%, model sparsity: 100%, test accuracy: 0.8100 at epoch 11
# Iteration 0 Citeseer | Graph sparsity 100%, model sparsity: 100%, test accuracy: 0.7090 at epoch 0
# Iteration 0 Pubmed | Graph sparsity 100%, model sparsity: 100%, test accuracy: 0.7620 at epoch 21

teacher_baseline = {
    'cora': 0.8100,
    'citeseer': 0.7090,
    'pubmed': 0.7620
}

In [14]:
graph_spar = np.ones(21)
model_spar = np.ones(21)
for i in range(1, 21):
    graph_spar[i] = graph_spar[i-1] - graph_spar[i-1] * 0.05 
    model_spar[i] = model_spar[i-1] - model_spar[i-1] * 0.2

graph_spar = (1 - graph_spar) * 100
model_spar = (1 - model_spar) * 100

In [15]:
model_spar

array([ 0.        , 20.        , 36.        , 48.8       , 59.04      ,
       67.232     , 73.7856    , 79.02848   , 83.222784  , 86.5782272 ,
       89.26258176, 91.41006541, 93.12805233, 94.50244186, 95.60195349,
       96.48156279, 97.18525023, 97.74820019, 98.19856015, 98.55884812,
       98.8470785 ])

In [76]:
from matplotlib.ticker import FormatStrFormatter, PercentFormatter

matplotlib.rcParams.update({'font.size': 22})
fig, axs = plt.subplots(3, constrained_layout=True, sharex='all', figsize=(12, 10))

lims = {
    'cora': [75, 83],
    'citeseer': [65, 73],
    'pubmed': [75, 82],
}
for i, dataset in enumerate(datasets):
    ax = axs[i]
    cora = stats_arr[dataset]

    kd_teacher, kd_f1, kd_loss = cora['kd']
    teacher_mean = np.mean(kd_teacher, axis=-1)
    teacher_std = np.std(kd_teacher, axis=-1)

    # fig, ax = plt.subplots(constrained_layout=True)
    ax.grid()
    ax.axhline(y = teacher_baseline[dataset] * 100, color = 'black', linestyle = '--')
    ax.plot(graph_spar, teacher_mean * 100, color='blue')
    ax.set_yticks(list(range(70, 85, 2)))
    ax.set_ylabel(f'{dataset.capitalize()} (\%)')
    ax.set_ylim(lims[dataset])
    ax.legend(['Baseline', 'Iterative pruning'])

ax = axs[2]
ax.set_xticks(graph_spar)
ax.set_xlabel('Graph sparsity (\%)')

ax2 = ax.twiny()

ax.tick_params(axis='x', rotation=45)
ax2.tick_params(axis='x', rotation=45)
ax2.plot(graph_spar, teacher_mean, color='blue')
ax2.set_xticks(graph_spar)
ax2.set_xticklabels("%.2f" % spar for spar in model_spar)
ax2.set_xlabel('Model sparsity (\%)')
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
# ax.tick_params(axis='y', which='major', labelsize=16)

ax2.xaxis.set_ticks_position('bottom') # set the position of the second x-axis to bottom
ax2.xaxis.set_label_position('bottom') # set the position of the second x-axis to bottom
ax2.spines['bottom'].set_position(('outward', 80))

# ax.set_ylim([75, 82])

# ax.set_title('Cora', fontdict={'fontsize':22})

plt.show()
plt.savefig('all_teachers')

/var/folders/1k/wlzqgvv50vd1tkj3jxbkk0m80000gn/T/ipykernel_22466/1999394296.py:51: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [26]:
kd_teacher, kd_f1, kd_loss = stats_arr['pubmed']['kd']
kd_f1_mean = np.mean(kd_f1, axis=-1)
kd_f1_std = np.std(kd_f1, axis=-1)

print(kd_f1_std)

[0.00309122 0.00531246 0.00286745 0.00659965 0.00618242 0.00169968
 0.00188562 0.00244948 0.00081651 0.00355901 0.00216025 0.001633
 0.00989952 0.00828653 0.00509903 0.00758654 0.00555778 0.00374166
 0.00637704 0.00478422 0.00188562]


In [77]:
from matplotlib.ticker import FormatStrFormatter, PercentFormatter

matplotlib.rcParams.update({'font.size': 20})
fig, axs = plt.subplots(3, constrained_layout=True, sharex='all', figsize=(12, 15))

lims = {
    'cora': [72, 82],
    'citeseer': [62, 72],
    'pubmed': [72, 82],
}
for i, dataset in enumerate(datasets):
    ax = axs[i]
    cora = stats_arr[dataset]

    kd_teacher, kd_f1, kd_loss = cora['kd']
    kd_f1_mean = np.mean(kd_f1, axis=-1)
    kd_f1_std = np.std(kd_f1 * 100, axis=-1)

    mi_teacher, mi_f1, mi_loss = cora['mi']
    mi_f1_mean = np.mean(mi_f1, axis=-1)
    mi_f1_std = np.std(mi_f1 * 100, axis=-1)

    full_f1, full_loss = cora['full']
    student_base_mean = np.mean(full_f1, axis=-1)
    student_base_std = np.std(full_f1 * 100, axis=-1)

    # fig, ax = plt.subplots(constrained_layout=True)
    ax.grid()
    # ax.axhline(y = student_base_mean * 100, color = 'black', linestyle = '--')
    ax.axhline(y = teacher_baseline[dataset] * 100, color = 'blue', linestyle = '--')
    ax.axhspan(student_base_mean * 100 - student_base_std, student_base_mean * 100 + student_base_std, facecolor='red', alpha=0.3)
    ax.errorbar(graph_spar, kd_f1_mean * 100, color='purple', yerr=kd_f1_std)
    # ax.errorbar(graph_spar, mi_f1_mean * 100, color='purple', yerr=mi_f1_std)
    # ax.set_yticks(list(range(70, 85, 2)))
    ax.set_ylabel(f'{dataset.capitalize()} (\%)')
    ax.set_ylim(lims[dataset])
    ax.legend(['Teacher baseline', 'Student baseline', 'KD-student'])

ax = axs[2]
ax.set_xticks(graph_spar)
ax.set_xlabel('Graph sparsity (\%)')

ax2 = ax.twiny()

ax.tick_params(axis='x', rotation=45)
ax2.tick_params(axis='x', rotation=45)
ax2.plot(graph_spar, kd_f1_mean, color='blue')
ax2.set_xticks(graph_spar)
ax2.set_xticklabels("%.2f" % spar for spar in model_spar)
ax2.set_xlabel('Model sparsity (\%)')
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
# ax.tick_params(axis='y', which='major', labelsize=16)

ax2.xaxis.set_ticks_position('bottom') # set the position of the second x-axis to bottom
ax2.xaxis.set_label_position('bottom') # set the position of the second x-axis to bottom
ax2.spines['bottom'].set_position(('outward', 80))

# ax.set_ylim([75, 82])

# ax.set_title('Cora', fontdict={'fontsize':22})

plt.show()
plt.savefig('kd-students')

/var/folders/1k/wlzqgvv50vd1tkj3jxbkk0m80000gn/T/ipykernel_22466/2284886104.py:62: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [78]:
from matplotlib.ticker import FormatStrFormatter, PercentFormatter

matplotlib.rcParams.update({'font.size': 20})
fig, axs = plt.subplots(3, constrained_layout=True, sharex='all', figsize=(12, 15))

lims = {
    'cora': [72, 82],
    'citeseer': [62, 72],
    'pubmed': [72, 82],
}
for i, dataset in enumerate(datasets):
    ax = axs[i]
    cora = stats_arr[dataset]

    kd_teacher, kd_f1, kd_loss = cora['kd']
    kd_f1_mean = np.mean(kd_f1, axis=-1)
    kd_f1_std = np.std(kd_f1 * 100, axis=-1)

    mi_teacher, mi_f1, mi_loss = cora['mi']
    mi_f1_mean = np.mean(mi_f1, axis=-1)
    mi_f1_std = np.std(mi_f1 * 100, axis=-1)

    full_f1, full_loss = cora['full']
    student_base_mean = np.mean(full_f1, axis=-1)
    student_base_std = np.std(full_f1 * 100, axis=-1)

    # fig, ax = plt.subplots(constrained_layout=True)
    ax.grid()
    # ax.axhline(y = student_base_mean * 100, color = 'black', linestyle = '--')
    ax.axhline(y = teacher_baseline[dataset] * 100, color = 'blue', linestyle = '--')
    ax.axhspan(student_base_mean * 100 - student_base_std, student_base_mean * 100 + student_base_std, facecolor='red', alpha=0.3)
    ax.errorbar(graph_spar, mi_f1_mean * 100, color='brown', yerr=mi_f1_std)
    # ax.set_yticks(list(range(70, 85, 2)))
    ax.set_ylabel(f'{dataset.capitalize()} (\%)')
    ax.set_ylim(lims[dataset])
    ax.legend(['Teacher baseline', 'Student baseline', 'LSP-student'])

ax = axs[2]
ax.set_xticks(graph_spar)
ax.set_xlabel('Graph sparsity (\%)')

ax2 = ax.twiny()

ax.tick_params(axis='x', rotation=45)
ax2.tick_params(axis='x', rotation=45)
ax2.plot(graph_spar, kd_f1_mean, color='blue')
ax2.set_xticks(graph_spar)
ax2.set_xticklabels("%.2f" % spar for spar in model_spar)
ax2.set_xlabel('Model sparsity (\%)')
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
# ax.tick_params(axis='y', which='major', labelsize=16)

ax2.xaxis.set_ticks_position('bottom') # set the position of the second x-axis to bottom
ax2.xaxis.set_label_position('bottom') # set the position of the second x-axis to bottom
ax2.spines['bottom'].set_position(('outward', 80))

# ax.set_ylim([75, 82])

# ax.set_title('Cora', fontdict={'fontsize':22})

plt.show()
plt.savefig('lsp-students')

/var/folders/1k/wlzqgvv50vd1tkj3jxbkk0m80000gn/T/ipykernel_22466/4183386059.py:61: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [62]:
print(mi_f1_mean)
print(kd_f1_mean)

[0.76933336 0.7713334  0.7636666  0.76533335 0.7696667  0.7723333
 0.7716667  0.77       0.77       0.76500005 0.766      0.761
 0.76500005 0.75799996 0.75699997 0.7586667  0.76566666 0.76399994
 0.75600004 0.7636667  0.7683334 ]
[0.76933336 0.7713334  0.7636666  0.76533335 0.7696667  0.7723333
 0.7716667  0.77       0.77       0.76500005 0.766      0.761
 0.76500005 0.75799996 0.75699997 0.7586667  0.76566666 0.76399994
 0.75600004 0.7636667  0.7683334 ]


In [111]:
kd_teacher, kd_f1, kd_loss = stats_arr['pubmed']['mi']

In [112]:
for it in [0, 7, 8, 19]:
    print('${:.2f}_{{\\pm{:.3f}}}$'.format(np.mean(kd_f1[it] * 100), np.std(kd_f1[it] * 100)))


$74.60_{\pm0.748}$
$74.93_{\pm0.834}$
$74.37_{\pm0.419}$
$74.57_{\pm0.450}$


In [121]:
f1, loss = stats_arr['pubmed']['full']
print('${:.2f}_{{\pm {:.3f}}}$'.format(np.mean(f1 * 100), np.std(f1 * 100)))

$75.07_{\pm 0.660}$


array([0.758, 0.772, 0.765], dtype=float32)